# Make a choropleth map using plotly and geojson regions


(optional) connect to google drive

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# plotly.py may be installed using pip...
#!pip install plotly==4.8.1


#*********************
#JupyterLab Support (Python 3.5+)
#For use in JupyterLab, install the jupyterlab and ipywidgets packages using pip...
#!pip install jupyterlab "ipywidgets==7.5"
#Then run the following commands to install the required JupyterLab extensions (note that this will require node to be installed):

# Basic JupyterLab renderer support
#!jupyter labextension install jupyterlab-plotly@4.8.1
# OPTIONAL: Jupyter widgets extension for FigureWidget support
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget@4.8.1

#*********************
#Static Image Export
#plotly.py supports static image export using the to_image and write_image functions in the plotly.io package. This functionality requires the installation of the plotly orca command line utility and the psutil Python package.
#!pip install psutil

#*********************
#Extended Geo Support
#Some plotly.py features rely on fairly large geographic shape files. The county choropleth figure factory is one such example. These shape files are distributed as a separate plotly-geo package. This package can be installed using pip...
#!pip install plotly-geo==1.0.0

     |████████████████████████████████| 11.5MB 49kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 7.6MB 24kB/s 
     |████████████████████████████████| 122kB 44.7MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
  Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
  Found existing installation: ipywidgets 7.5.1
    Uninstalling ipywidgets-7.5.1:
      Successfully uninstalled ipywidgets-7.5.1


An error occured.
ValueError: Please install nodejs >=10.0.0 before continuing. nodejs may be installed using conda or directly from the nodejs website.
See the log file for details:  /tmp/jupyterlab-debug-uqnls9fy.log
An error occured.
ValueError: Please install nodejs >=10.0.0 before continuing. nodejs may be installed using conda or directly from the nodejs website.
See the log file for details:  /tmp/jupyterlab-debug-f4ur2ak7.log
     |████████████████████████████████| 23.7MB 1.5MB/s 


In [0]:
#!pip install fuzzywuzzy

In [0]:
# Chart Studio support
#The chart-studio package can be used to upload plotly figures to Plotly's Chart Studio Cloud or On-Prem service. This package can be installed using pip...

# !pip install chart_studio
#!pip install chart-studio==1.0.0

Import Libraries

In [1]:
import json
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process
from matplotlib.colors import Normalize
from matplotlib import cm
from itertools import product
import copy

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)


import chart_studio.plotly as py
import plotly.graph_objs as go

MAPBOX_APIKEY = "pk.eyJ1IjoiYW5hYmVsY2Fyb2wiLCJhIjoiY2p5NHU5aWE5MDBuaDNtcTA1Z25ndmJ4YSJ9.6uDUV1_ZSXY9clXNpPraUQ"
# %matplotlib inline

ModuleNotFoundError: No module named 'fuzzywuzzy'

# Environmental Indicators

Getting the data

# Differentiated Waste Collection

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/ProduzioneRD_regioni_2017.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

Load the geojson

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

Count the number of provinces and get the province name

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

Center of each province

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

Match province names with dataframe metric

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

reindex and rename the dataframe

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

Make the sources:

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

Define the colours, the colorscale and the hover text

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " [Kg/citizen*year] "
    with_data = '<b>{}</b> <br> {} Differentiated Waste'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

### Ready to make map

Run all the functions

In [ ]:
colormap = 'Blues'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

The scattermapbox data

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

The borders

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

The layout

In [ ]:
data_url = "https://www.catasto-rifiuti.isprambiente.it/index.php?pg=&width=1440&height=900"

layout = dict(title="Differentiated Waste Collection (2017)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "ISPRA Ambiente",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
#fig = dict(data=[data], layout=layout)
#iplot(fig)

fig=go.Figure(data=[data], layout=layout)
fig.show()

which gives the plot below ( that I am rendering from an iframe from my plotly account) 

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# Urban Waste Collection

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/ProduzioneRU_regioni_2017.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Regioni'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))
 

def get_centers():
    lon, lat =[], []
    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']
        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])
        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.
    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]
    return {key: value for (key, value) in zip(list1, matched)}
match_dict = match_regions(un.index, province_names)

df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()


def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources


def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]
def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))
    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " [Kg/citizen*year] "
    with_data = '<b>{}</b> <br> {} Municipality Waste'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]



colormap = 'Blues'
cmin = df_reindexed.min()
cmax = df_reindexed.max()
sources = make_sources(downsample=10)
lons, lats = get_centers()
sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)
tickformat = ".0"

data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +
        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

data_url = "https://www.catasto-rifiuti.isprambiente.it/index.php?pg=&width=1440&height=900"
layout = dict(title="Urban Waste (2017) <br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "ISPRA Ambiente",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,
      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

fig = dict(data=[data], layout=layout)
iplot(fig)

#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')



# Licensed Vehicles

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/motorizzazioni1.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Province'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/province.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_PRO'] for k in range(n_provinces)]
print("there are {} provinces ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)
print(match_dict)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " "
    with_data = '<b>{}</b> <br> {} Licensed Vehicles(in thousands)'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Blues'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://www.datiopen.it/it/opendata/Tasso_di_motorizzazione_per_i_comuni_capoluogo_di_provincia_per_anno"

layout = dict(title="Licensed Vehicles (in thousands) <br> " + 
                    """using <a href={}>open data</a> by """.format(data_url) +
                      "Il portale Italiano dell'Open Data",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~Anabelcarol/16.embed"></iframe>')

# Picnic Areas

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/AreePicnic_Province_2008-2016.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'province'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/province.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_PRO'] for k in range(n_provinces)]
print("there are {} provinces ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)
print (match_dict)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + "%"
    with_data = '<b>{}</b> <br> {} Picnic Areas'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Greens'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://www.datiopen.it/it/opendata/Mappa_delle_aree_picnic_in_Italia/"

layout = dict(title="Picnic Areas in Italy (2008-2016) <br> " + 
                    """using <a href={}>open data</a> by """.format(data_url) +
                      "Il portale Italiano dell'Open Data ",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')


# Swimming Pools

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/SwimmingPool_2011-2016.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'province'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/province.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_PRO'] for k in range(n_provinces)]
print("there are {} provinces ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)
print(match_dict)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + "  "
    with_data = '<b>{}</b> <br> {} Swimming Pools'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Greens'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://www.datiopen.it/it/opendata/Mappa_delle_piscine_in_Italia"

layout = dict(title="Swimming Pools in Italy (2016) <br> " + 
                    """using <a href={}>open data</a> by """.format(data_url) +
                      "Il portale Italiano dell'Open Data",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# Tv transmitters

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/Italy_TV_transmitters.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'province'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/province.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_PRO'] for k in range(n_provinces)]
print("there are {} provinces ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)
print(match_dict)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + "%"
    with_data = '<b>{}</b> <br> {} TV Transmitters'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Blues'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://www.datiopen.it/en/opendata/Mappa_delle_antenne_in_Italia"

layout = dict(title="TV transmitters (2008-2016) <br> " + 
                    """using <a href={}>open data</a> by  """.format(data_url) +
                      "Il portale Italiano dell'Open Data",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# No internet No broad

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/Internet_no_broadcast_connexion_noavailable_2016_2.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Regioni'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " Families "
    with_data = '<b>{}</b> <br> {} with No Internet'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it/Index.aspx?DataSetCode=DCIS_BIBLIOT"

layout = dict(title="2016- Families with No Internet-No Broadcast Connexion-Italy <br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# No Internet High Cost

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/internet_no_for_HighCost_2016_regioni.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Regioni'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " families "
    with_data = '<b>{}</b> <br> {} with No Internet'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it/Index.aspx?DataSetCode=DCIS_BIBLIOT"

layout = dict(title="No Internet-High Cost Connection (2016)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# Seats in libraries

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/Biblioteche_regioni_2010_1.csv', index_col=0)
un = un['Value']/10000.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)
print(match_dict)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " "
    with_data = '<b>{}</b> <br> {} Seats in libraries'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it/Index.aspx?DataSetCode=DCIS_BIBLIOT"

layout = dict(title="Number of seats for Readers in Public Libraries (2010)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# How workers reach place of work - public transport

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/StradaCasaLavoro_2017-2018_usano_mezziTrasporto.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " citizens (in thousands) "
    with_data = '<b>{}</b> <br> {} using Public Transport'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Citizens (in thousands) Public Transport (2017-2018) <br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# How workers reach place of work - underground

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/StradaCasaLavoro_2017-2018_use-underor-overground.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " citizens(in thousands) "
    with_data = '<b>{}</b> <br> {} using Underground'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Citizens (in thousands) using Underground (2017-2018)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

# How workers reach place of work - bycicle

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/StradaCasaLavoro_2017-2018_usano_bicycle.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " citizens (in thousands) "
    with_data = '<b>{}</b> <br> {} using Bicycle'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it/"

layout = dict(title="Citizens(in thousands) using Bicycle (2017-2018) <br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

# How Students reach their Schools - public transport

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/StradaCasaScuola_2017-2018_PublicTransport.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " students (in thousands) "
    with_data = '<b>{}</b> <br> {} using Public Transport to School'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Students(in thousands) using Public Transport (2017-2018)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

# How Students reach their Schools - distance<15min

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/StradaCasaScuola_2017-2018_vicinanza.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " students (in thousands) "
    with_data = '<b>{}</b> <br> {} less than 15min to school'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Students (in thousands)-less than 15 min to School (2017-2018)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# How Students reach their Schools - private vehicle

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/StradaCasaScuola_2017-2018_vehicle.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " students (in thousands) "
    with_data = '<b>{}</b> <br> {} using Private Vehicle'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Oranges'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Students(in thousands)- using Private Vehicle (2017-2018) <br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# classic music and opera

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/Classic_music_Shows_Opera_2018.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " shows "
    with_data = '<b>{}</b> <br> {} Classic music and Opera 2018'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Greens'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Classic music Shows and Opera (2018)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# discotheque and concerts

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/discotheque_Festivals-Concerts_2018.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " shows "
    with_data = '<b>{}</b> <br> {} discotheque Festivals and Concerts 2018'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Greens'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Discotheque Festivals Concerts (2018)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# Archeological sites and museums

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/Museums_Archaeological-sites_2018.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " shows "
    with_data = '<b>{}</b> <br> {} Museums and Archaeological-sites 2018'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'Greens'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Museums and Archaeological-sites (2018)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')

# violence Against Women

In [ ]:
un = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/AbuseViolence_Against_Women2014_1_6.csv', index_col=0)
un = un['Value']/100.
un.index.name = 'Region'
un.head()

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Choropleth_Italy/regioni.geojson') as f:
     geojson = json.load(f)

In [ ]:
n_provinces = len(geojson['features'])
province_names = [geojson['features'][k]['properties']['NOME_REG'] for k in range(n_provinces)]
print("there are {} regions ".format(n_provinces))

In [ ]:
def get_centers():
    lon, lat =[], []

    for k in range(n_provinces):
        geometry = geojson['features'][k]['geometry']

        if geometry['type'] == 'Polygon':
            coords=np.array(geometry['coordinates'][0])
        elif geometry['type'] == 'MultiPolygon':
            coords=np.array(geometry['coordinates'][0][0])

        lon.append(sum(coords[:,0]) / len(coords[:,0]))
        lat.append(sum(coords[:,1]) / len(coords[:,1]))
            
    return lon, lat

In [ ]:
def match_regions(list1, list2):
    """
    Match the region names of self.metric with the region names of the geojson
    using fuzzy matching.

    Return a dict with {'old_name': 'new_name'}
    """
    matched = [process.extract(list1[i], list2, limit=1, scorer=fuzz.partial_ratio)[0][0] for i in range(0,len(list1))]

    return {key: value for (key, value) in zip(list1, matched)}

match_dict = match_regions(un.index, province_names)

In [ ]:
df_tmp = un.copy()
df_tmp.index = df_tmp.index.map(match_dict) # rename
df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)] # drop duplicates
                                                      # from fuzzy match gone wrong
#give the same index order as the geojson
df_reindexed = df_tmp.reindex(index = province_names)
df_reindexed.head()

In [ ]:
def make_sources(downsample = 10):
    sources = []
    geojson_copy = copy.deepcopy(geojson['features'])
    
    for feature in geojson_copy:
        
        if downsample > 0:
            coords = np.array(feature['geometry']['coordinates'][0][0])
            coords = coords[::downsample]
            feature['geometry']['coordinates'] = [[coords]]
            
        sources.append(dict(type = 'FeatureCollection', 
                            features = [feature])
                      )
    return sources

In [ ]:
def scalarmappable(cmap, cmin, cmax):
        colormap = cm.get_cmap(cmap)
        norm = Normalize(vmin=cmin, vmax=cmax)
        return cm.ScalarMappable(norm=norm, cmap=colormap)
    
def get_scatter_colors(sm, df):
    grey = 'rgba(128,128,128,1)'
    return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]

def get_colorscale(sm, df, cmin, cmax):
    xrange = np.linspace(0, 1, len(df))
    values = np.linspace(cmin, cmax, len(df))

    return [[i, 'rgba' + str(sm.to_rgba(v, bytes = True))] for i,v in zip(xrange, values) ]
 
def get_hover_text(df) :
    text_value = (df*100).round(2).astype(str) + " "
    with_data = '<b>{}</b> <br> {} Abuse/Violence Against Women'
    no_data = '<b>{}</b> <br> no data'
    
    return [with_data.format(p,v) if v != 'nan%' else no_data.format(p) for p,v in zip(df.index, text_value)]

In [ ]:
colormap = 'RdPu'
cmin = df_reindexed.min()
cmax = df_reindexed.max()

sources = make_sources(downsample=10)
lons, lats = get_centers()

sm = scalarmappable(colormap, cmin, cmax)
scatter_colors = get_scatter_colors(sm, df_reindexed)
colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
hover_text = get_hover_text(df_reindexed)

tickformat = ".0%"

In [ ]:
data = dict(type='scattermapbox',
             lat=lats,
             lon=lons,
             mode='markers',
             text=hover_text,
             marker=dict(size=1,
                         color=scatter_colors,
                         showscale = True,
                         cmin = df_reindexed.min(),
                         cmax = df_reindexed.max(),
                         colorscale = colorscale,
                         colorbar = dict(tickformat = tickformat )
                         ),
             showlegend=False,
             hoverinfo='text'
             )

In [ ]:
layers=([dict(sourcetype = 'geojson',
        source =sources[k],
        below="",
        type = 'line',
        line = dict(width = 1),
        color = 'black',
        ) for k in range(n_provinces)] +

        [dict(sourcetype = 'geojson',
             source =sources[k],
             below="water",
             type = 'fill',
             color = scatter_colors[k],
             opacity=0.8
            ) for k in range(n_provinces)]
        )

In [ ]:
data_url = "http://dati.istat.it"

layout = dict(title="Abuse/Violence Against Women (2014)<br> " + 
                    """using <a href={}>open data</a> by the """.format(data_url) +
                      "Italian National Institute of Statistics",
      autosize=False,
      width=700,
      height=800,
      hovermode='closest',
      hoverdistance = 30,

      mapbox=dict(accesstoken=MAPBOX_APIKEY,
                  layers=layers,
                  bearing=0,
                  center=dict(
                  lat=41.871941,
                  lon=12.567380),
                  pitch=0,
                  zoom=4.9,
                  style = 'light'
            )
      )

In [ ]:
fig = dict(data=[data], layout=layout)
iplot(fig)

In [ ]:
#from IPython.display import HTML
#HTML('<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~vincenzo.pota/24.embed"></iframe>')